In [2]:
import random as rnd
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
##LENDO OS DADOS 
data = pd.read_csv('semeion.csv', delimiter = " ", header = None)


In [4]:
### SEPARANDO EM X (COVARIÁVEIS) E Y (RESPOSTA)
X = data.iloc[:,0:256]
Y = data.iloc[:, -11:-2]


In [5]:
X= np.array(X)
Y = np.array(Y)

In [54]:
### FUNÇÃO QUE FAZ A REDE RBF 
def RBF(X_train, Y_train,X_test, sigma):
    ##PADRONIZANDO
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    ##DETERMINANDO OS CENTROS USANDO KMEANS
    K_cent = 230
    km = KMeans(n_clusters = K_cent, max_iter = 100)
    km.fit(X_train)
    cent = km.cluster_centers_
    

    #### MATRIZ G
    shape = X_train.shape
    row = shape[0]
    column = K_cent
    G = np.empty((row,column),dtype = float)
    for i in range(row):
        for j in range(column):
            dist = np.linalg.norm(X_train[i] - cent[j])
            G[i][j] = math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))
    
    #### MATRIZ W PESOS
    GTG= np.dot(G.T,G)
    GTG_inv= np.linalg.inv(GTG)
    fac= np.dot(GTG_inv,G.T)
    W= np.dot(fac,Y_train)
    
    ### MATRIZ G DE TESTE
    row= X_test.shape[0]
    column= K_cent
    G_test= np.empty((row,column), dtype= float)
    for i in range(row):
        for j in range(column):
            dist= np.linalg.norm(X_test[i]-cent[j])
            G_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))
    
    ### CALCULANDO A PREDIÇÃO
    prediction= np.dot(G_test,W)
    prediction= 0.5*(np.sign(prediction-0.5)+1)
    
    return prediction
    
    

### A seguir aplicamos a rede nos dados usando k-fold com k=10. 

In [73]:
###Aplicação
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
vac = []
for i, (train_index, test_index) in enumerate (kf.split(X, Y.argmax(1))):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    prediction = RBF(x_train,y_train,x_test,sigma = 8)
    score= accuracy_score(prediction,y_test)
    vac.append(score)
    print(score)
    
print ('Vetor de acurácias', vac)
       

0.8650306748466258
0.9006211180124224
0.85625
0.89375
0.86875
0.8553459119496856
0.9182389937106918
0.8616352201257862
0.8924050632911392
0.9025974025974026
Vetor de acurácias de cada foldo [0.8650306748466258, 0.9006211180124224, 0.85625, 0.89375, 0.86875, 0.8553459119496856, 0.9182389937106918, 0.8616352201257862, 0.8924050632911392, 0.9025974025974026]


In [75]:
## Calculando a acurácia média

mean_accuracy=np.mean(vac)
print('Acurácia média da RBF',mean_accuracy)

Acurácia média da RBF 0.8814624384533755


## Testando uma MLP para comparação

In [1]:
## USANDO A BIBLIOTECA MLPCLASSIFIER E APLICANDO USANDO KFOLD
from sklearn.neural_network import MLPClassifier
vac2=[]
for i, (train_index, test_index) in enumerate (kf.split(X, Y.argmax(1))):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index] 
    clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1,30), random_state=1, max_iter = 500)
    clf.fit(x_train, y_train)
    y_pred2 = clf.predict(x_test)
    precision2 = accuracy_score(y_test, y_pred2)
    vac2.append(precision2)
print(vac2)

ImportError: cannot import name 'datasets'

In [76]:
### CALCULANDO A ACURÁCIA MÉDIA
mean_precision2=np.mean(vac2)

print('Acurácia Média da MLP',mean_precision2)

Acurácia Média da MLP 0.8700387666085462
